In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words=open('../names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
chars=sorted(list(set(''.join(words))))
stoi={s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos={i:s for s,i in stoi.items()}
print(stoi.items())

dict_items([('a', 1), ('b', 2), ('c', 3), ('d', 4), ('e', 5), ('f', 6), ('g', 7), ('h', 8), ('i', 9), ('j', 10), ('k', 11), ('l', 12), ('m', 13), ('n', 14), ('o', 15), ('p', 16), ('q', 17), ('r', 18), ('s', 19), ('t', 20), ('u', 21), ('v', 22), ('w', 23), ('x', 24), ('y', 25), ('z', 26), ('.', 0)])


In [6]:
#context_length
block_size=3
X,Y=[],[]
for w in words[:5]:
    context=[0]*block_size
    for ch in w+'.':
        ix=stoi[ch]
        X.append(context.copy())
        Y.append(ix)
        context=context[1:]+[ix]

X=torch.tensor(X)
Y=torch.tensor(Y)
print(X)

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1],
        [ 0,  0,  0],
        [ 0,  0, 15],
        [ 0, 15, 12],
        [15, 12,  9],
        [12,  9, 22],
        [ 9, 22,  9],
        [22,  9,  1],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, 22],
        [ 1, 22,  1],
        [ 0,  0,  0],
        [ 0,  0,  9],
        [ 0,  9, 19],
        [ 9, 19,  1],
        [19,  1,  2],
        [ 1,  2,  5],
        [ 2,  5, 12],
        [ 5, 12, 12],
        [12, 12,  1],
        [ 0,  0,  0],
        [ 0,  0, 19],
        [ 0, 19, 15],
        [19, 15, 16],
        [15, 16,  8],
        [16,  8,  9],
        [ 8,  9,  1]])


In [7]:
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.emb=torch.nn.Embedding(len(stoi), 16)
        self.fc=torch.nn.Linear(16*block_size, len(stoi))
    def forward(self, x):
        x=self.emb(x)
        x=x.view(x.size(0), -1)
        x=self.fc(x)
        return x
    
model=MLP()
model(X)

tensor([[ 9.8821e-01,  3.5393e-01,  3.6358e-01, -2.9492e-01,  7.5729e-01,
         -3.2866e-01,  1.2694e+00, -7.5176e-02,  5.8986e-01,  4.3887e-02,
         -1.0616e-01, -5.2960e-01,  5.1899e-01, -9.0448e-03, -2.1059e-01,
          4.7867e-01, -5.5718e-01, -4.8495e-01, -1.1224e-01, -3.4626e-01,
          4.1104e-01,  7.8926e-02, -2.8760e-01, -2.5881e-01, -1.8873e-02,
          2.2583e-01, -1.8266e-01],
        [-2.0930e-01, -3.6843e-01,  2.3506e-01, -2.8160e-01,  9.8793e-01,
          1.7668e-01, -5.4481e-01,  1.0670e-01,  1.5496e-01,  8.8064e-01,
         -9.0568e-01, -7.0237e-01,  8.2826e-01,  7.4246e-01, -8.9654e-01,
         -1.9668e-01,  4.0519e-01, -6.7666e-01,  1.6446e-01, -4.6877e-01,
          1.3921e-01,  1.4745e-01, -3.3714e-01,  1.6254e-02,  7.2794e-01,
          4.3205e-01,  1.0088e-01],
        [ 3.8643e-01, -5.2793e-01,  1.0302e+00, -1.7141e-02,  3.9920e-01,
         -1.6236e-01, -9.3671e-01,  1.3728e+00, -3.0205e-01, -1.7263e-01,
          8.7668e-01,  2.8267e-01, -4.51

In [12]:
C=torch.randn((27,2))
#F.one_hot(torch.tensor(5),27).float()@C

C[X]


tensor([[[ 1.1667,  0.1642],
         [ 1.1667,  0.1642],
         [ 1.1667,  0.1642]],

        [[ 1.1667,  0.1642],
         [ 1.1667,  0.1642],
         [-0.0432,  0.5110]],

        [[ 1.1667,  0.1642],
         [-0.0432,  0.5110],
         [ 0.6263, -1.1047]],

        [[-0.0432,  0.5110],
         [ 0.6263, -1.1047],
         [ 0.6263, -1.1047]],

        [[ 0.6263, -1.1047],
         [ 0.6263, -1.1047],
         [-0.1892,  0.2184]],

        [[ 1.1667,  0.1642],
         [ 1.1667,  0.1642],
         [ 1.1667,  0.1642]],

        [[ 1.1667,  0.1642],
         [ 1.1667,  0.1642],
         [ 2.2352, -0.3175]],

        [[ 1.1667,  0.1642],
         [ 2.2352, -0.3175],
         [-0.6874, -1.9393]],

        [[ 2.2352, -0.3175],
         [-0.6874, -1.9393],
         [-0.0105,  0.1201]],

        [[-0.6874, -1.9393],
         [-0.0105,  0.1201],
         [-0.7076,  1.6625]],

        [[-0.0105,  0.1201],
         [-0.7076,  1.6625],
         [-0.0105,  0.1201]],

        [[-0.7076,  1

In [15]:
emb=C[X]
print(emb.shape)
W1=torch.randn((6,100))
b1=torch.randn((100,))

#torch.cat(torch.unbind(emb,1),1)
a=torch.arange(18)
a.view(9,2)

a.storage()


torch.Size([32, 3, 2])


/tmp/ipykernel_469979/3192258440.py:10: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  a.storage()


 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
 12
 13
 14
 15
 16
 17
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 18]